In [ ]:
%load_ext autoreload
%autoreload 2
import cv2
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.io
import torch as t
import torchvision
from torchvision.transforms import functional as F
from utils.config import opt
from model import FasterRCNNVGG16
from trainer import FasterRCNNTrainer
from data.util import  read_image
from utils.vis_tool import vis_bbox
from utils import array_tool as at
from PIL import Image, ImageDraw
%matplotlib inline

In [ ]:
faster_rcnn = FasterRCNNVGG16()
trainer = FasterRCNNTrainer(faster_rcnn).cuda()

In [3]:
def calculate_iou(box1, box2):
    # Unpack the coordinates from the bounding boxes
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    # Determine the coordinates of the intersection rectangle
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    # Calculate the area of intersection
    inter_width = max(0, inter_x_max - inter_x_min)
    inter_height = max(0, inter_y_max - inter_y_min)
    inter_area = inter_width * inter_height

    # Calculate the area of both bounding boxes
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)

    # Calculate the area of the union
    union_area = box1_area + box2_area - inter_area

    # Calculate the Intersection over Union (IoU)
    iou = inter_area / union_area if union_area > 0 else 0

    return iou

# Load the weights
trainer.load('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/pretrained_weights/fasterrcnn_12211511_0.701052458187_torchvision_pretrain.pth')
opt.caffe_pretrain=False # this model was trained from torchvision-pretrained model

# Initialize an empty list to hold the query info data
lst_data = []

# Open the file and read line by line
with open('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/prw/query_info.txt', 'r') as file:
    for line in file:
        # Split the line into individual elements
        elements = line.strip().split()
        
        # Convert numeric elements to appropriate data types
        elements[0] = int(elements[0])  # Convert the first element to int
        elements[1:5] = [int(float(x)) for x in elements[1:5]]  # Convert the next four elements to int
        #xmin, ymin, width, height = elements[1], elements[2], elements[3], elements[4]
        #y2, x2 = ymin + height, xmin + width
        
        # Append the list of elements to the data list
        lst_data.append(elements)

# Ensure that the output folder exists
output_folder = './query_prw_tog_p_fgsm_distractors_iou'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all .jpg files in the input directory
input_folder = './frames_tog_p_fgsm'
image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

not_detected_images = []  # List to store names of images with no detections

# Load the distractor image (make sure this is a valid path to your distractor image)
distractor_input_folder = './datasets/prw/query_box'
distractor_image_path = './assets/example_2.png'
distractor_image = Image.open(distractor_image_path)

iou_not_found = 0

for image_file in image_files:
    input_img_path = os.path.join(input_folder, image_file)
    output_img_path = os.path.join(output_folder, image_file)
        
    # Load the image for detection
    img = read_image(input_img_path)
    img = t.from_numpy(img)[None]
    
    # Load the image for bounding boxes
    img_bboxes = cv2.imread(input_img_path)
    
    # Perform detection
    try:
        _bboxes, _labels, _scores = trainer.faster_rcnn.predict(img,visualize=True)       
    
        # Convert predictions to numpy
        bboxes = at.tonumpy(_bboxes[0])
        labels = at.tonumpy(_labels[0]).reshape(-1)
        scores = at.tonumpy(_scores[0]).reshape(-1)
            
        # Filter predictions to show only "person" class
        person_class_id = 14
        person_indices = labels == person_class_id
        person_bboxes = bboxes[person_indices]
        person_labels = labels[person_indices]
        person_scores = scores[person_indices]
    
        #xmin, ymin, width, height = lst_data[0][1], lst_data[0][2], lst_data[0][3], lst_data[0][4]
        #y2, x2 = ymin + height, xmin + width
        #cropped_image_annotation = img_bboxes[ymin:y2, xmin:x2]
        #cropped_image_annotation_coordinates = [ymin, xmin, y2, x2]
        #cv2.imwrite(output_img_path + '_' + str(i) + '_annotation_' + '.jpg', cropped_image_annotation)
    
        for annotation in lst_data:
            if annotation[5] in str(input_img_path):
                xmin, ymin, width, height = annotation[1], annotation[2], annotation[3], annotation[4]
                y2, x2 = ymin + height, xmin + width
                cropped_image_annotation = img_bboxes[ymin:y2, xmin:x2]
                cropped_image_annotation_coordinates = [ymin, xmin, y2, x2]
                distractor_img = Image.open(distractor_input_folder + '/' + str(annotation[0]) + '_' + input_img_path.split('/')[-1])

                if len(bboxes) == 0:
                    resized_distractor = distractor_image.resize(distractor_img.size, Image.Resampling.LANCZOS)
                    resized_distractor.save(output_folder + '/' + str(annotation[0]) + '_' + annotation[5] + '_00'  + '.jpg', format='JPEG')
                    not_detected_images.append(str(annotation[0]) + '_' + image_file[:-4] + '_00' + '.jpg')
                for bbox in bboxes:
                    xmin, ymin, xmax, ymax = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
                    cropped_image_detector = [xmin, ymin, xmax, ymax]
                    cropped_image_detector = img_bboxes[xmin:xmax, ymin:ymax]
                    cropped_image_detector_coordinates = [xmin, ymin, xmax, ymax]
            
                    # Calculate IoU
                    iou = calculate_iou(cropped_image_annotation_coordinates, cropped_image_detector_coordinates)
                    
                    # Check if IoU is greater than or equal to 0.5
                    if iou >= 0.5:
                        cv2.imwrite(output_folder + '/' + str(annotation[0]) + '_' + annotation[5] + '_00' + '.jpg', cropped_image_detector)
                        break
                    else:
                        iou_not_found = iou_not_found + 1
                        if iou_not_found == len(bboxes):
                            resized_distractor = distractor_image.resize(distractor_img.size, Image.Resampling.LANCZOS)
                            resized_distractor.save(output_folder + '/' + str(annotation[0]) + '_' + annotation[5] + '_00' + '.jpg', format='JPEG')
                            not_detected_images.append(str(annotation[0]) + '_' + image_file[:-4] + '_00' + '.jpg')
                        pass
                iou_not_found = 0
    except:
        pass

# Write the list of not detected images to a file
with open('query_not_detected.txt', 'w') as file:
    for image_name in not_detected_images:
        file.write(image_name + '\n')

In [ ]:
# Define paths
folder_path = './query_prw_iou'
not_detected_file = './query_prw_iou_not_detected.txt'

# Read the file names from the txt file
with open(not_detected_file, 'r') as file:
    lines = file.readlines()

# Clean the file names (strip any surrounding whitespace or newline characters)
file_names_to_delete = {line.strip() for line in lines}

# List all files in the distractor folder
files_in_folder = os.listdir(folder_path)

# Delete files that match the exact names in the txt file
for file_name in files_in_folder:
    if file_name in file_names_to_delete:
        file_path = os.path.join(folder_path, file_name)
        os.remove(file_path)
        print(f"Deleted: {file_path}")

In [ ]:
# Define the paths to the folders
query_folder = './query_prw_iou'
bounding_box_folder = './bounding_box_test_prw_iou'

# Get a set of the identity prefixes from the query folder
query_prefixes = {file_name[:3] for file_name in os.listdir(query_folder)}

# List all files in the bounding box folder
files_in_bounding_box = os.listdir(bounding_box_folder)

# Loop through each file in the bounding box folder
for file_name in files_in_bounding_box:
    # Extract the first three characters (identity prefix) from the file name
    file_prefix = file_name[:3]
    
    # Check if the prefix is not in the query folder's prefixes
    if file_prefix not in query_prefixes:
        # Construct the full path to the file
        file_path = os.path.join(bounding_box_folder, file_name)
        
        # Delete the file
        os.remove(file_path)
        print(f"Deleted: {file_path}")

In [9]:
# Load the weights
trainer.load('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/pretrained_weights/fasterrcnn_12211511_0.701052458187_torchvision_pretrain.pth')
opt.caffe_pretrain=False # this model was trained from torchvision-pretrained model

# Ensure that the output folder exists
output_folder = './query_prw_distractor_no_iou'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all .jpg files in the input directory
input_folder = './query_prw_no_iou'
image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

detected_images = []  # List to store names of images with detections

# Load the distractor image (make sure this is a valid path to your distractor image)
distractor_image_path = './assets/example_2.png'
distractor_image = Image.open(distractor_image_path)

for image_file in image_files:
    input_img_path = os.path.join(input_folder, image_file)
    output_img_path = os.path.join(output_folder, image_file)

    # Load the image
    img = read_image(input_img_path)
    img = t.from_numpy(img)[None]

    # Load the image for distractor
    distractor_img = Image.open(input_img_path)

    # Perform detection
    try:
        _bboxes, _labels, _scores = trainer.faster_rcnn.predict(img,visualize=True)       

        # Convert predictions to numpy
        bboxes = at.tonumpy(_bboxes[0])
        labels = at.tonumpy(_labels[0]).reshape(-1)
        scores = at.tonumpy(_scores[0]).reshape(-1)
        
        # # Filter predictions to show only "person" class
        person_class_id = 14
        person_indices = labels == person_class_id
        person_bboxes = bboxes[person_indices]
        person_labels = labels[person_indices]
        person_scores = scores[person_indices]
        
        # Check if there are any detections; if not, create and save a resized distractor image
        if len(person_bboxes) == 0:
            # Resize the distractor image to match the size of the input image
            resized_distractor = distractor_image.resize(distractor_img.size, Image.Resampling.LANCZOS)
            # Save the resized distractor image as a JPG
            resized_distractor.save(output_img_path, format='JPEG')
        else:
            # Save the original image if detections were found
            detected_images.append(image_file)
            normalized_tensor = (img - img.min()) / (img.max() - img.min())
            torchvision.utils.save_image(normalized_tensor, output_img_path, format='JPEG')
    except:
        # Save the distractor image as a JPG
        distractor_img.save(output_img_path, format='JPEG')
        
# Write the list of detected images to a file
with open('detected_query_iou_distractor.txt', 'w') as file:
    for image_name in detected_images:
        file.write(image_name + '\n')

In [ ]:
trainer.load('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/pretrained_weights/fasterrcnn_12211511_0.701052458187_torchvision_pretrain.pth')
opt.caffe_pretrain=False # this model was trained from torchvision-pretrained model
_bboxes, _labels, _scores = trainer.faster_rcnn.predict(img,visualize=True)
# vis_bbox(at.tonumpy(img[0]),
#          at.tonumpy(_bboxes[0]),
#          at.tonumpy(_labels[0]).reshape(-1),
#          at.tonumpy(_scores[0]).reshape(-1))
# it failed to find the dog, but if you set threshold from 0.7 to 0.6, you'll find it

# Debug: Print all detected labels and their scores
# print("Detected labels and scores:")
# for label, score in zip(labels, scores):
#     print(f"Label: {label}, Score: {score}")

# Convert predictions to numpy
bboxes = at.tonumpy(_bboxes[0])
labels = at.tonumpy(_labels[0]).reshape(-1)
scores = at.tonumpy(_scores[0]).reshape(-1)

# # Filter predictions to show only "person" class
person_class_id = 14
person_indices = labels == person_class_id
person_bboxes = bboxes[person_indices]
person_labels = labels[person_indices]
person_scores = scores[person_indices]

# # Visualize only "person" class predictions
vis_bbox(at.tonumpy(img[0]),
         person_bboxes,
         person_labels,
         person_scores)

# Save the image
#save_path = '/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/cropped_imgs/orl_without_bboxes.jpg'  # Replace with your desired path
#normalized_tensor = (img - img.min()) / (img.max() - img.min())
#torchvision.utils.save_image(normalized_tensor, save_path, format='jpeg')

# Load the image
image = cv2.imread('misc/s15538.jpg')

# Define the "person" bounding boxes
bounding_boxes = person_bboxes

# Convert bounding box coordinates to integers
bounding_boxes = [[int(coord) for coord in box] for box in bounding_boxes]

# Draw the bounding boxes on the image
for i, box in enumerate(bounding_boxes):
    x1, y1, x2, y2 = box
    #cv2.rectangle(image, (y1, x1), (y2, x2), (0, 255, 0), 2)  # (0, 255, 0) is the color (green), 2 is the thickness

    # Crop the images within the bounding boxes
    cropped_image = image[x1:x2, y1:y2]
    
    # Save the cropped images
    cv2.imwrite(f'/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/cropped_imgs/cropped_image_{i}.jpg', cropped_image)

    print("{}: {}, {}, {}, {}".format(i, y1, x1, y2, x2))

In [ ]:
mat_train = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/test/train_test/Train.mat')

data_train = mat_train['Train']

lst_persons = []
lst_image = []
lst_images = []
lst_coordinates = []

for sample in data_train: 
    #print(sample[0][0][0][0]) # Persons
    id_person = int(str(sample[0][0][0][0]).replace('p', '').replace('[', '').replace("'", '').replace(']', ''))
    lst_persons.append(id_person)
    for sample_2 in sample[0][0][0][2][0]:
        #print(sample_2[0]) # Images of person
        name_image = str(sample_2[0]).replace('[', '').replace("'", '').replace(']', '')
        lst_image.append(name_image)
        #print(sample_2[1]) # Coordinates
        number_coordinates = str(sample_2[1]).replace('[', '').replace("'", '').replace(']', '').replace('  ', ' ')
        number_coordinates = number_coordinates.split(" ")
        lst_coordinates.append(number_coordinates)
    lst_images.append(lst_image)
    lst_image = []

#print(lst_persons)
#print(lst_image)
#print(lst_images)
#print(lst_coordinates)
#total_elements = sum(len(sublist) for sublist in lst_images)
#print(total_elements)

#i = 0
j = 0
k = 0
z = 1
#flag_empty_coordinate = 0
counter_number_persons_with_empty_bounding_boxes = 0

for sample in lst_persons:
    name_person = sample
    for sample_2 in lst_images[j]:
        image = cv2.imread(f'datasets/cuhk_sysu/Image/SSM/{sample_2}')
        name_image_cleaner = sample_2.replace('s', '')
        if lst_coordinates[k][0] != '' and lst_coordinates[k][1] != '' and lst_coordinates[k][2] != '' and lst_coordinates[k][3] != '':
            xmin, ymin, width, height = int(lst_coordinates[k][0]), int(lst_coordinates[k][1]), int(lst_coordinates[k][2]), int(lst_coordinates[k][3])
            y2, x2 = ymin + height, xmin + width
            cropped_image = image[ymin:y2, xmin:x2]
            cv2.imwrite(f'datasets/cuhk_sysu/bounding_box_train/{name_person}_c{z}s1_000000_{name_image_cleaner}', cropped_image)
            z = z + 1
        #else:
            #print(name_person, end=',') # Persons with some empty coordinate in an image (not being considered)
            #flag_empty_coordinate = 1
        k = k + 1
    if z == 1:
        counter_number_persons_with_empty_bounding_boxes = counter_number_persons_with_empty_bounding_boxes + 1
    j = j + 1
    z = 1
    #if flag_empty_coordinate == 1:
        #directory = '/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/bounding_box_train/'
        #for filename in os.listdir(directory):
            #if filename.startswith(str(name_person)):
                #file_path = os.path.join(directory, filename)
                #os.remove(file_path)
        #flag_empty_coordinate = 0
    #else:
        #i = i + 1
    #if i == 751:
        #break

print(counter_number_persons_with_empty_bounding_boxes)

In [ ]:
mat_query = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/test/train_test/TestG100.mat')

data_query = mat_query['TestG100']

lst_persons = []
lst_persons_with_empty_coordinates = []
lst_image = []
lst_coordinates = []

for sample in data_query[0]:
     #print(sample[0][0][0][0][3]]) # Persons
    id_person = int(str(sample[0][0][0][3]).replace('p', '').replace('[', '').replace("'", '').replace(']', ''))
    lst_persons.append(id_person)
    for sample_2 in sample[0][0]:
        #print(sample_2[0]) # Image of persons
        name_image = str(sample_2[0]).replace('[', '').replace("'", '').replace(']', '')
        lst_image.append(name_image)
        #print(sample_2[1]) # Coordinates
        number_coordinates = str(sample_2[1]).replace('[', '').replace("'", '').replace(']', '').replace('  ', ' ')
        number_coordinates = number_coordinates.split(" ")
        lst_coordinates.append(number_coordinates)

#print(lst_persons)
#print(lst_image)
#print(lst_coordinates)

# i = 0
j = 0
# k = 0
# z = 1
# flag_empty_coordinate = 0
counter_number_persons_with_empty_bounding_boxes = 0

for sample in lst_persons:
    name_person = sample
    name_image = lst_image[j]
    image = cv2.imread(f'datasets/cuhk_sysu/Image/SSM/{name_image}')
    name_image_cleaner = name_image.replace('s', '')
    if lst_coordinates[j][0] != '' and lst_coordinates[j][1] != '' and lst_coordinates[j][2] != '' and lst_coordinates[j][3] != '':
        xmin, ymin, width, height = int(lst_coordinates[j][0]), int(lst_coordinates[j][1]), int(lst_coordinates[j][2]), int(lst_coordinates[j][3])
        y2, x2 = ymin + height, xmin + width
        cropped_image = image[ymin:y2, xmin:x2]
        cv2.imwrite(f'datasets/cuhk_sysu/query/{name_person}_c1s1_000000_{name_image_cleaner}', cropped_image)
    #z = z + 1
    #else:
        #print(name_person, end=',') # Persons with some empty coordinate in an image (not being considered)
        #flag_empty_coordinate = 1
    #k = k + 1
    #if z == 1:
    else:
        lst_persons_with_empty_coordinates.append(name_person)
        counter_number_persons_with_empty_bounding_boxes = counter_number_persons_with_empty_bounding_boxes + 1
    j = j + 1
    #z = 1
    #if flag_empty_coordinate == 1:
        #directory = '/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/query/'
        #for filename in os.listdir(directory):
            #if filename.startswith(str(name_person)):
                #file_path = os.path.join(directory, filename)
                #os.remove(file_path)
        #flag_empty_coordinate = 0
    #else:
        #i = i + 1
    #if i == 750:
        #break

lst_persons_test = [item for item in lst_persons if item not in lst_persons_with_empty_coordinates]

print(counter_number_persons_with_empty_bounding_boxes)
print(len(lst_persons_test))

In [ ]:
mat_test = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/test/train_test/TestG100.mat')

data_test = mat_test['TestG100']

lst_persons_with_empty_coordinates = []
dict_person_images = {}
dict_image_coordinates = {}
dict_person_images_coordinates = {}

for sample in data_test[0]:
    id_person = str(sample[0][0][0][3]).replace('p', '').replace('[', '').replace("'", '').replace(']', '')
    dict_person_images[id_person] = []
    dict_person_images_coordinates[id_person] = []
    for sample_2 in sample[1][0]:
        sample_coordinate = sample_2[1]
        if sample_coordinate.size != 0:
            name_image = str(sample_2[0]).replace('[', '').replace("'", '').replace(']', '')
            dict_image_coordinates[name_image] = []
            dict_person_images[id_person].append(name_image)
            number_coordinates = str(sample_coordinate).replace('[', '').replace("'", '').replace(']', '').replace('  ', ' ')
            number_coordinates = number_coordinates.split(" ")
            dict_image_coordinates[name_image].append(number_coordinates)
            dict_person_images_coordinates[id_person].append(name_image)
            dict_person_images_coordinates[id_person].append(number_coordinates)

#print(dict_person_images)
#print(dict_image_coordinates)
#print(dict_person_images_coordinates)
#total_elements = sum(len(sublist) for sublist in lst_images)
#print(total_elements)

i = 0
j = 0
#k = 0
z = 2
#flag_empty_coordinate = 0
counter_number_persons_with_empty_bounding_boxes = 0

for sample in lst_persons_test:
    name_person = sample
    for key, value in dict_person_images_coordinates.items():
        if int(key) == name_person:
            j = len(value)
            while j != 0:
                name_image = value[i]
                image = cv2.imread(f'datasets/cuhk_sysu/Image/SSM/{name_image}')
                name_image_cleaner = name_image.replace('s', '')
                if value[i+1][0] != '' and value[i+1][1] != '' and value[i+1][2] != '' and value[i+1][3] != '':
                    xmin, ymin, width, height = int(value[i+1][0]), int(value[i+1][1]), int(value[i+1][2]), int(value[i+1][3])
                    y2, x2 = ymin + height, xmin + width
                    cropped_image = image[ymin:y2, xmin:x2]
                    cv2.imwrite(f'datasets/cuhk_sysu/bounding_box_test/{name_person}_c{z}s1_000000_{name_image_cleaner}', cropped_image)
                    z = z + 1
                i = i + 2
                j = j - 2
            if z == 2:
                counter_number_persons_with_empty_bounding_boxes = counter_number_persons_with_empty_bounding_boxes + 1
                lst_persons_with_empty_coordinates.append(name_person)
    i = 0
    j = 0
    z = 2

directory = '/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/query/'
for sample_2 in lst_persons_with_empty_coordinates:
    name_person = sample_2
    for filename in os.listdir(directory):
        if filename.startswith(str(name_person) + '_'):
            file_path = os.path.join(directory, filename)
            os.remove(file_path)

lst_persons_final = [item for item in lst_persons_test if item not in lst_persons_with_empty_coordinates]

print(counter_number_persons_with_empty_bounding_boxes)
print(len(lst_persons_final))

In [ ]:
# Load pool MAT file
pool_mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/pool.mat')

pool_data = pool_mat['pool']

lst_pool_imgs = []

for sample in pool_data:
    sample_img = sample[0]
    img_name = str(sample_img).replace('[', '').replace("'", '').replace(']', '')
    lst_pool_imgs.append(img_name)

# Load TestG100 MAT file
testg100_mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/test/train_test/TestG100.mat')

testg100_data = testg100_mat['TestG100']

lst_testg100_imgs = []
lst_testg100_persons = []

for sample in testg100_data[0]:
    sample_img = sample[0][0][0]
    sample_person = sample[0][0]
    img_name = str(sample_img[0]).replace('[', '').replace("'", '').replace(']', '')
    person_identity = str(sample_img[3]).replace('p', '').replace('[', '').replace("'", '').replace(']', '')
    lst_testg100_imgs.append(img_name)
    lst_testg100_persons.append(person_identity)
    

lst_imgs = [item for item in lst_pool_imgs if item not in lst_testg100_imgs]

dict_testg100_imgs = {}

for sample in testg100_data[0]:
    sample_img = sample[0][0][0]
    person_identity = str(sample_img[3]).replace('p', '').replace('[', '').replace("'", '').replace(']', '')
    dict_testg100_imgs[person_identity] = []
    for sample_2 in sample[1][0]:
        sample_coordinate = sample_2[1]
        if sample_coordinate.size != 0:
            sample_img_2 = sample_2[0]
            img_name = str(sample_img_2).replace('[', '').replace("'", '').replace(']', '')
            dict_testg100_imgs[person_identity].append(img_name)

#dict_testg100_imgs = {key: ', '.join(value) for key, value in dict_testg100_imgs.items()}

#print(len(lst_imgs))
#print(lst_testg100_imgs[0])
#print(lst_testg100_persons[0])
#print(lst_imgs[0])
#print(dict_testg100_imgs)
#first_key = next(iter(dict_testg100_imgs))
#first_elements = dict_testg100_imgs[first_key]
#print(f"First key: {first_key}")
#print("Elements:", first_elements)

#def print_duplicates(input_list):
    #i = 0
    #seen = set()
    #duplicates = set()
    #for item in input_list:
        #if item in seen:
            #duplicates.add(item)
        #seen.add(item)
    #for duplicate in duplicates:
        #print(duplicate)
        #i = i + 1
    #print(i)

# Example usage:
#print_duplicates(lst_testg100_imgs)
#print(len(lst_imgs) - 167)

# # Display all variable names contained in the file
#print(mat.keys())

# Print header and version information
#print("Header:", mat['__header__'])
#print("Version:", mat['__version__'])
#print("Globals:", mat['__globals__'])

# Access and print data in 'TestG50'
#testg100_data = mat['TestG100']
#print("TestG100 Data:", testg100_data)

trainer.load('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/pretrained_weights/fasterrcnn_12211511_0.701052458187_torchvision_pretrain.pth')
opt.caffe_pretrain=False # this model was trained from torchvision-pretrained model

#directory = 'datasets/cuhk_sysu/Image/SSM/'
#lst_imgs_path = []

#for root, dirs, files in os.walk(directory):
    #for file in files:
        #img_path = os.path.join(root, file)
        #lst_imgs_path.append(img_path)

#lst_imgs_path = [path.split('SSM/')[1] for path in lst_imgs_path]

# Loop through each sample
#for sample in testg100_data[0]:  # Assuming testg100g_data[0] to get to the actual data if it's structured with an extra dimension
    #sample_img = sample[0][0][0]  # Image
for sample in lst_imgs:
    #sample_img = sample  # Image
    #for sample_2 in sample_img:
        #sample_img_2 = sample_2
    #sample_img_2 = sample[0][0][0][1]  # Location
    #sample_img_3 = sample[1][0][1][1]  # Location
    #print(str(sample_img) + str(sample_img_2) + str(sample_img_3))
    
    #print(str(sample_img))
    #print(str(sample_img[3]))
    #lst_person.append(str(sample_img[3]))
    #i = i + 1

    #img_name = str(sample_img[1][0][1][0]).replace('[', '').replace("'", '').replace(']', '')

    #img_name = str(sample_img[0]).replace('[', '').replace("'", '').replace(']', '')

    #if img_name in lst_imgs_path:
       #print(len(lst_imgs_path))
        
        #pass
    
    #else:

        #print("b")

    img_name = sample

    img_name_cleaner = img_name.replace('s', '')

    img = read_image(f'datasets/cuhk_sysu/Image/SSM/{img_name}')
    
    image = cv2.imread(f'datasets/cuhk_sysu/Image/SSM/{img_name}')
    
    img = t.from_numpy(img)[None]
    
    _bboxes, _labels, _scores = trainer.faster_rcnn.predict(img,visualize=True)
    
    bboxes = at.tonumpy(_bboxes[0])
    labels = at.tonumpy(_labels[0]).reshape(-1)
    scores = at.tonumpy(_scores[0]).reshape(-1)
        
    person_class_id = 14
    person_indices = labels == person_class_id
    person_bboxes = bboxes[person_indices]
    person_labels = labels[person_indices]
    person_scores = scores[person_indices]
    
    bounding_boxes = person_bboxes
    
    bounding_boxes = [[int(coordinate) for coordinate in box] for box in bounding_boxes]
    
    for i, box in enumerate(bounding_boxes):
        x1, y1, x2, y2 = box
        cropped_image = image[x1:x2, y1:y2]
        for key, value in dict_testg100_imgs.items():
            if img_name in value:
                person_identity = key
                cv2.imwrite(f'datasets/cuhk_sysu/bounding_box_test/{person_identity}_c1s1_000000_0{img_name_cleaner}', cropped_image)
            #else:
                #cv2.imwrite(f'datasets/cuhk_sysu/bounding_box_test/00000_c1s1_000000_0{img_name_cleaner}', cropped_image)
                #break

    #print(str(sample_img[1][0][1][0]))
    #print(str(sample_img[1][0][1][1]))
    #print(str(sample_img[1]))
    #print("{}, {}".format(str(sample_img[0]), str(sample_img[1])))

#lst_person_cleaned = [s.replace('[', '').replace(']', '').replace("'", '').replace('"', '') for s in lst_person]

#unique_set = set(lst_person_cleaned)

# Check for duplicates
#if len(unique_set) != len(lst_person_cleaned):
    #print("There are duplicate elements.")
#else:
    #print("All elements are unique.")

#print(sorted(lst_person_cleaned))
#print(i)

In [ ]:
# Load pool MAT file
pool_mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/pool.mat')

pool_data = pool_mat['pool']

lst_pool_imgs = []

for sample in pool_data:
    sample_img = sample[0]
    img_name = str(sample_img).replace('[', '').replace("'", '').replace(']', '')
    lst_pool_imgs.append(img_name)

# Load TestG100 MAT file
testg100_mat = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/test/train_test/TestG100.mat')

testg100_data = testg100_mat['TestG100']

lst_testg100_imgs = []
lst_testg100_persons = []

for sample in testg100_data[0]:
    sample_img = sample[0][0][0]
    sample_person = sample[0][0]
    img_name = str(sample_img[0]).replace('[', '').replace("'", '').replace(']', '')
    person_identity = str(sample_img[3]).replace('p', '').replace('[', '').replace("'", '').replace(']', '')
    lst_testg100_imgs.append(img_name)
    lst_testg100_persons.append(person_identity)
    

lst_imgs = [item for item in lst_pool_imgs if item not in lst_testg100_imgs]

dict_testg100_imgs = {}

for sample in testg100_data[0]:
    sample_img = sample[0][0][0]
    person_identity = str(sample_img[3]).replace('p', '').replace('[', '').replace("'", '').replace(']', '')
    dict_testg100_imgs[person_identity] = []
    for sample_2 in sample[1][0]:
        sample_coordinate = sample_2[1]
        if sample_coordinate.size != 0:
            sample_img_2 = sample_2[0]
            img_name = str(sample_img_2).replace('[', '').replace("'", '').replace(']', '')
            dict_testg100_imgs[person_identity].append(img_name)

#dict_testg100_imgs = {key: ', '.join(value) for key, value in dict_testg100_imgs.items()}

#print(len(lst_imgs))
#print(lst_testg100_imgs[0])
#print(lst_testg100_persons[0])
#print(lst_imgs[0])
#print(dict_testg100_imgs)
#first_key = next(iter(dict_testg100_imgs))
#first_elements = dict_testg100_imgs[first_key]
#print(f"First key: {first_key}")
#print("Elements:", first_elements)

#def print_duplicates(input_list):
    #i = 0
    #seen = set()
    #duplicates = set()
    #for item in input_list:
        #if item in seen:
            #duplicates.add(item)
        #seen.add(item)
    #for duplicate in duplicates:
        #print(duplicate)
        #i = i + 1
    #print(i)

# Example usage:
#print_duplicates(lst_testg100_imgs)
#print(len(lst_imgs) - 167)

# # Display all variable names contained in the file
#print(mat.keys())

# Print header and version information
#print("Header:", mat['__header__'])
#print("Version:", mat['__version__'])
#print("Globals:", mat['__globals__'])

# Access and print data in 'TestG50'
#testg100_data = mat['TestG100']
#print("TestG100 Data:", testg100_data)

trainer.load('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/pretrained_weights/fasterrcnn_12211511_0.701052458187_torchvision_pretrain.pth')
opt.caffe_pretrain=False # this model was trained from torchvision-pretrained model

#directory = 'datasets/cuhk_sysu/Image/SSM/'
#lst_imgs_path = []

#for root, dirs, files in os.walk(directory):
    #for file in files:
        #img_path = os.path.join(root, file)
        #lst_imgs_path.append(img_path)

#lst_imgs_path = [path.split('SSM/')[1] for path in lst_imgs_path]

# Loop through each sample
#for sample in testg100_data[0]:  # Assuming testg100g_data[0] to get to the actual data if it's structured with an extra dimension
    #sample_img = sample[0][0][0]  # Image
for sample in lst_imgs:
    #sample_img = sample  # Image
    #for sample_2 in sample_img:
        #sample_img_2 = sample_2
    #sample_img_2 = sample[0][0][0][1]  # Location
    #sample_img_3 = sample[1][0][1][1]  # Location
    #print(str(sample_img) + str(sample_img_2) + str(sample_img_3))
    
    #print(str(sample_img))
    #print(str(sample_img[3]))
    #lst_person.append(str(sample_img[3]))
    #i = i + 1

    #img_name = str(sample_img[1][0][1][0]).replace('[', '').replace("'", '').replace(']', '')

    #img_name = str(sample_img[0]).replace('[', '').replace("'", '').replace(']', '')

    #if img_name in lst_imgs_path:
       #print(len(lst_imgs_path))
        
        #pass
    
    #else:

        #print("b")

    img_name = sample

    img_name_cleaner = img_name.replace('s', '')

    img = read_image(f'datasets/cuhk_sysu/Image/SSM/{img_name}')
    
    image = cv2.imread(f'datasets/cuhk_sysu/Image/SSM/{img_name}')
    
    img = t.from_numpy(img)[None]
    
    _bboxes, _labels, _scores = trainer.faster_rcnn.predict(img,visualize=True)
    
    bboxes = at.tonumpy(_bboxes[0])
    labels = at.tonumpy(_labels[0]).reshape(-1)
    scores = at.tonumpy(_scores[0]).reshape(-1)
        
    person_class_id = 14
    person_indices = labels == person_class_id
    person_bboxes = bboxes[person_indices]
    person_labels = labels[person_indices]
    person_scores = scores[person_indices]
    
    bounding_boxes = person_bboxes
    
    bounding_boxes = [[int(coordinate) for coordinate in box] for box in bounding_boxes]
    
    for i, box in enumerate(bounding_boxes):
        x1, y1, x2, y2 = box
        cropped_image = image[x1:x2, y1:y2]
        for key, value in dict_testg100_imgs.items():
            if img_name in value:
                person_identity = key
                cv2.imwrite(f'datasets/cuhk_sysu/bounding_box_test/{person_identity}_c1s1_000000_0{img_name_cleaner}', cropped_image)
            #else:
                #cv2.imwrite(f'datasets/cuhk_sysu/bounding_box_test/00000_c1s1_000000_0{img_name_cleaner}', cropped_image)
                #break

    #print(str(sample_img[1][0][1][0]))
    #print(str(sample_img[1][0][1][1]))
    #print(str(sample_img[1]))
    #print("{}, {}".format(str(sample_img[0]), str(sample_img[1])))

#lst_person_cleaned = [s.replace('[', '').replace(']', '').replace("'", '').replace('"', '') for s in lst_person]

#unique_set = set(lst_person_cleaned)

# Check for duplicates
#if len(unique_set) != len(lst_person_cleaned):
    #print("There are duplicate elements.")
#else:
    #print("All elements are unique.")

#print(sorted(lst_person_cleaned))
#print(i)

In [6]:
def calculate_iou(box1, box2):
    # Unpack the coordinates from the bounding boxes
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    # Determine the coordinates of the intersection rectangle
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    # Calculate the area of intersection
    inter_width = max(0, inter_x_max - inter_x_min)
    inter_height = max(0, inter_y_max - inter_y_min)
    inter_area = inter_width * inter_height

    # Calculate the area of both bounding boxes
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)

    # Calculate the area of the union
    union_area = box1_area + box2_area - inter_area

    # Calculate the Intersection over Union (IoU)
    iou = inter_area / union_area if union_area > 0 else 0

    return iou

# Load the weights
trainer.load('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/pretrained_weights/fasterrcnn_12211511_0.701052458187_torchvision_pretrain.pth')
opt.caffe_pretrain=False # this model was trained from torchvision-pretrained model

mat_query = scipy.io.loadmat('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/annotation/test/train_test/TestG100.mat')

data_query = mat_query['TestG100']

lst_persons = []
lst_persons_with_empty_coordinates = []
lst_image = []
lst_coordinates = []

for sample in data_query[0]:
     #print(sample[0][0][0][0][3]]) # Persons
    id_person = int(str(sample[0][0][0][3]).replace('p', '').replace('[', '').replace("'", '').replace(']', ''))
    lst_persons.append(id_person)
    for sample_2 in sample[0][0]:
        #print(sample_2[0]) # Image of persons
        name_image = str(sample_2[0]).replace('[', '').replace("'", '').replace(']', '')
        lst_image.append(name_image)
        #print(sample_2[1]) # Coordinates
        number_coordinates = str(sample_2[1]).replace('[', '').replace("'", '').replace(']', '').replace('  ', ' ')
        number_coordinates = number_coordinates.split(" ")
        lst_coordinates.append(number_coordinates)

# Ensure that the output folder exists
output_folder = './query_cuhk_sysu_tog_p_fgsm_distractors_iou'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all .jpg files in the input directory
input_folder = './SSM_tog_p_fgsm'
image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

not_detected_images = []  # List to store names of images with no detections

# Load the distractor image (make sure this is a valid path to your distractor image)
distractor_input_folder = '/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/query_cuhk_sysu_no_iou'
distractor_image_path = './assets/example_2.png'
distractor_image = Image.open(distractor_image_path)

j = 0
iou_not_found = 0

# Open the file for writing
with open('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/query_info.txt', 'w') as file:
    # Iterate through the lists and write each line
    for sample in lst_persons:
        name_person = sample
        name_image = lst_image[j]
        image = cv2.imread(f'datasets/cuhk_sysu/Image/SSM/{name_image}')
        name_image_cleaner = name_image.replace('s', '')

        if lst_coordinates[j][0] != '' and lst_coordinates[j][1] != '' and lst_coordinates[j][2] != '' and lst_coordinates[j][3] != '':
            # Format the line
            line = f"{str(sample)} {str(lst_coordinates[j][0])} {str(lst_coordinates[j][1])} {str(lst_coordinates[j][2])} {str(lst_coordinates[j][3])} {str(name_image)}\n"
            
            # Write the line to the file
            file.write(line)

        j = j + 1

# Initialize an empty list to hold the query info data
lst_data = []

file.close()

# Open the file and read line by line
with open('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/query_info.txt', 'r') as file:
    for line in file:
        # Split the line into individual elements
        elements = line.strip().split()
        
        # Convert numeric elements to appropriate data types
        elements[0] = int(elements[0])  # Convert the first element to int
        elements[1:5] = [int(x) for x in elements[1:5]]  # Convert the next four elements to int
        #xmin, ymin, width, height = elements[1], elements[2], elements[3], elements[4]
        #y2, x2 = ymin + height, xmin + width
        
        # Append the list of elements to the data list
        lst_data.append(elements)

#i = 0

for image_file in image_files:
    input_img_path = os.path.join(input_folder, image_file)
    output_img_path = os.path.join(output_folder, image_file)
        
    # Load the image for detection
    img = read_image(input_img_path)
    img = t.from_numpy(img)[None]
    
    # Load the image for bounding boxes
    img_bboxes = cv2.imread(input_img_path)
    
    # Perform detection
    try:
        _bboxes, _labels, _scores = trainer.faster_rcnn.predict(img,visualize=True)       
    
        # Convert predictions to numpy
        bboxes = at.tonumpy(_bboxes[0])
        labels = at.tonumpy(_labels[0]).reshape(-1)
        scores = at.tonumpy(_scores[0]).reshape(-1)
            
        # Filter predictions to show only "person" class
        person_class_id = 14
        person_indices = labels == person_class_id
        person_bboxes = bboxes[person_indices]
        person_labels = labels[person_indices]
        person_scores = scores[person_indices]
    
        for annotation in lst_data:
            if annotation[5] in str(input_img_path):
                xmin, ymin, width, height = annotation[1], annotation[2], annotation[3], annotation[4]
                y2, x2 = ymin + height, xmin + width
                cropped_image_annotation = img_bboxes[ymin:y2, xmin:x2]
                cropped_image_annotation_coordinates = [ymin, xmin, y2, x2]
                try:
                    distractor_img = Image.open(distractor_input_folder + '/' + str(annotation[0]) + '_c1s1_000000_' + input_img_path.split('/')[-1].replace('s', ''))
                except:
                    distractor_img = Image.open(distractor_input_folder + '/' + '8_c1s1_000000_12.jpg')

                if len(bboxes) == 0:
                    resized_distractor = distractor_image.resize(distractor_img.size, Image.Resampling.LANCZOS)
                    resized_distractor.save(output_folder + '/' + str(annotation[0]) + '_c1s1_000000_' + str(annotation[5]).replace('s', ''), format='JPEG')
                    not_detected_images.append(str(annotation[0]) + '_c1s1_000000_' + str(annotation[5]).replace('s', ''))
                for bbox in bboxes:
                    xmin, ymin, xmax, ymax = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
                    cropped_image_detector = [xmin, ymin, xmax, ymax]
                    cropped_image_detector = img_bboxes[xmin:xmax, ymin:ymax]
                    cropped_image_detector_coordinates = [xmin, ymin, xmax, ymax]
            
                    # Calculate IoU
                    iou = calculate_iou(cropped_image_annotation_coordinates, cropped_image_detector_coordinates)
                    
                    # Check if IoU is greater than or equal to 0.5
                    if iou >= 0.5:
                        cv2.imwrite(output_folder + '/' + str(annotation[0]) + '_c1s1_000000_' + str(annotation[5]).replace('s', ''), cropped_image_detector)
                        break
                    else:
                        iou_not_found = iou_not_found + 1
                        if iou_not_found == len(bboxes):
                            resized_distractor = distractor_image.resize(distractor_img.size, Image.Resampling.LANCZOS)
                            resized_distractor.save(output_folder + '/' + str(annotation[0]) + '_c1s1_000000_' + str(annotation[5]).replace('s', ''), format='JPEG')
                            not_detected_images.append(str(annotation[0]) + '_c1s1_000000_' + str(annotation[5]).replace('s', ''))
                        pass
                iou_not_found = 0
                #i = i + 1
        #if i == 5:
            #break
        #break
    except:
        pass

# Write the list of not detected images to a file
with open('./query_cuhk_sysu_tog_p_fgsm_distractors_iou_not_detected.txt', 'w') as file:
    for image_name in not_detected_images:
        file.write(image_name + '\n')

In [ ]:
# Define the paths to the folders
query_folder = './query_cuhk_sysu_tog_p_fgsm_distractors_iou'
bounding_box_folder = '/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/bounding_box_test'

# Get a set of the identity prefixes from the bounding box folder
bounding_box_prefixes = {file_name.split('_')[0] for file_name in os.listdir(bounding_box_folder)}

# List all files in the query folder
files_in_query = os.listdir(query_folder)

# Loop through each file in the query folder
for file_name in files_in_query:
    # Extract the first three characters (identity prefix) from the file name
    file_prefix = file_name.split('_')[0]
    
    # Check if the prefix is not in the bounding box folder's prefixes
    if file_prefix not in bounding_box_prefixes:
        # Construct the full path to the file
        file_path = os.path.join(query_folder, file_name)
        
        # Delete the file
        os.remove(file_path)
        print(f"Deleted: {file_path}")

In [ ]:
# Define paths
folder_path = './query_cuhk_sysu_p_fgsm_tog_distractors_iou'
not_detected_file = './query_cuhk_sysu_p_fgsm_tog_distractors_iou_not_detected.txt'

# Read the file names from the txt file
with open(not_detected_file, 'r') as file:
    lines = file.readlines()

# Clean the file names (strip any surrounding whitespace or newline characters)
file_names_to_delete = {line.strip() for line in lines}

# List all files in the distractor folder
files_in_folder = os.listdir(folder_path)

# Delete files that match the exact names in the txt file
for file_name in files_in_folder:
    if file_name in file_names_to_delete:
        file_path = os.path.join(folder_path, file_name)
        os.remove(file_path)
        print(f"Deleted: {file_path}")

In [ ]:
# Load the weights
trainer.load('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/pretrained_weights/fasterrcnn_12211511_0.701052458187_torchvision_pretrain.pth')
opt.caffe_pretrain=False # this model was trained from torchvision-pretrained model

# Load the image for detection
img = read_image('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/s15539.jpg')
img = t.from_numpy(img)[None]
img_bboxes = cv2.imread('/home/eduardoandrade/Phd/experiments/models/od/simple-faster-rcnn-pytorch/datasets/cuhk_sysu/s15539.jpg')
_bboxes, _labels, _scores = trainer.faster_rcnn.predict(img,visualize=True)       
bboxes = at.tonumpy(_bboxes[0])
labels = at.tonumpy(_labels[0]).reshape(-1)
scores = at.tonumpy(_scores[0]).reshape(-1)
person_class_id = 14
person_indices = labels == person_class_id
person_bboxes = bboxes[person_indices]
person_labels = labels[person_indices]
person_scores = scores[person_indices]
print(bboxes)